# Introduction

Intron annotation files I used can come from two sources: 

1. using leafcutter's leafviz/gtf2leafcutter.pl script
2. using gtftookits `gtftk` methods

See details of extracting annotations from gencode gtf files from either of the
two methods in snakefile (~/cdai/annotations/workflow/Snakefile)


## Conclusion

Leafcutter's `leafviz/gtf2leafcutter.pl` produces non-standard intron coordinates. Even though the produced file is using `.bed`
file extension, the file is not in correct BED format!

- The start position is in correct BED format. 
- The end position is WRONG! The reported end value from file is 1 greater than the correct BED formatted end value!

**use `gtftk`, do not use `gtf2leafcutter.pl`!**


## Details 

see below: ...

In [4]:
#| label: set-up

import os
import pandas as pd

In [10]:
os.getcwd()

'/project2/yangili1/cdai/annotations'

In [9]:
os.chdir('../../')

In [11]:
#| include: false

!ls -lahR hg38

hg38:
total 48M
drwxrwsr-x 4 chaodai pi-yangili1 4.0K May 12 14:50 .
drwxrwsr-x 6 chaodai pi-yangili1 4.0K May 12 17:57 ..
-rw-rw-r-- 1 chaodai pi-yangili1  48M May 12 12:25 gencode.v43.primary_assembly.annotation.gtf.gz
drwxrwsr-x 2 chaodai pi-yangili1 4.0K May 15 17:03 use_gtftk
drwxrwsr-x 2 chaodai pi-yangili1 4.0K May 12 18:24 use_leafviz

hg38/use_gtftk:
total 12M
drwxrwsr-x 2 chaodai pi-yangili1 4.0K May 15 17:03 .
drwxrwsr-x 4 chaodai pi-yangili1 4.0K May 12 14:50 ..
-rw-rw-r-- 1 chaodai pi-yangili1 4.7M May 12 18:15 gencode_v43_basic.exons.bed.gz
-rw-rw-r-- 1 chaodai pi-yangili1 1.2M May 14 21:16 gencode_v43_basic.genes.bed.gz
-rw-rw-r-- 1 chaodai pi-yangili1 4.4M May 15 17:04 gencode_v43_basic.intron_by_transcript.bed.gz
-rw-rw-r-- 1 chaodai pi-yangili1 1.6M May 12 18:44 gencode_v43_basic.transcript.bed.gz

hg38/use_leafviz:
total 21M
drwxrwsr-x 2 chaodai pi-yangili1 4.0K May 12 18:24 .
drwxrwsr-x 4 chaodai pi-yangili1 4.0K May 12 14:50 ..
-rw-rw-r-- 1 chaodai pi-yangili1    0

## Introns from gtftk

In [20]:
intron_gtftk = pd.read_csv('hg38/use_gtftk/gencode_v43_basic.intron_by_transcript.bed.gz', sep='\t', header=None, 
                           names=['chrom', 'start', 'end', 'name', 'score', 'strand'])

In [21]:
intron_gtftk.iloc[1:2]
intron_gtftk.shape

,chrom,start,end,name,score,strand
1,chr1,12721,13220,intron|ENSG00000290825.1|ENST00000456328.2|DDX...,2,+


(486009, 6)

In [22]:
intron_leaf = pd.read_csv('hg38/use_leafviz/gencode_v43_all_introns.bed.gz', sep='\t', header=None,
                         names=['chrom', 'start', 'end', 'gname', 'gid', 'strand', 'tid', 'score', 'gene_type', 'tags'])

In [23]:
intron_leaf.iloc[1:2,:]
intron_leaf.shape

,chrom,start,end,gname,gid,strand,tid,score,gene_type,tags
1,chrX,100635252,100635558,TSPAN6,ENSG00000000003.15,-,ENST00000373020.9,2,protein_coding,basic|Ensembl_canonical|MANE_Select|appris_pri...


(486009, 10)

In [37]:
intron_gtftk[intron_gtftk['name'].str.contains('ZBTB18')].sort_values(['name', 'start']).reset_index(drop=True)

,chrom,start,end,name,score,strand
0,chr1,244051444,244053787,intron|ENSG00000179456.12|ENST00000358704.4|ZB...,1,+
1,chr1,244048993,244053787,intron|ENSG00000179456.12|ENST00000622512.1|ZB...,1,+


## Introns from leafcutter perl script

In [38]:
intron_leaf[intron_leaf.gname.str.contains('ZBTB18')].sort_values(['tid', 'start']).reset_index(drop=True)

,chrom,start,end,gname,gid,strand,tid,score,gene_type,tags
0,chr1,244051444,244053788,ZBTB18,ENSG00000179456.12,+,ENST00000358704.4,1,protein_coding,basic|Ensembl_canonical|MANE_Select|appris_pri...
1,chr1,244048993,244053788,ZBTB18,ENSG00000179456.12,+,ENST00000622512.1,1,protein_coding,basic|appris_alternative_1


## Comparison notes: 

1. Notice for each gene, the number of introns from either method are the same. 
2. gtftk produced introns follow correct BED format, however, leafcutter produced results do not follow correct BEDformat. 
    - Specifically, leafcutter produced introns have the **same start coordinate, but an increase by 1 end coordinate**.

## Check gtftk produced intron junctions, along with transcript and exon coordiantes

1. example for ZBTB18 shows the intron for transcript 704.4 is from ...444(close) to ...787(open), while first exons ends at ...444(open) and the second
exon starts from ...787 (close). 
2. similarly the second intron is also correct, considering all transcript, exons, and introns are strictly BED format.

In [43]:
# transcripts
!zcat hg38/use_gtftk/gencode_v43_basic.transcript.bed.gz | awk '$4 ~ /ZBTB18/' | sort -k4 -k2

chr1	244051282	244057476	ENSG00000179456.12|ENST00000358704.4|ZBTB18|protein_coding	.	+
chr1	244048938	244057472	ENSG00000179456.12|ENST00000622512.1|ZBTB18|protein_coding	.	+


In [44]:
# exons
!zcat hg38/use_gtftk/gencode_v43_basic.exons.bed.gz | awk '$4 ~ /ZBTB18/' | sort -k4 -k2

chr1	244051282	244051444	ENSG00000179456.12|ENST00000358704.4|ZBTB18|protein_coding	.	+
chr1	244053787	244057476	ENSG00000179456.12|ENST00000358704.4|ZBTB18|protein_coding	.	+
chr1	244048938	244048993	ENSG00000179456.12|ENST00000622512.1|ZBTB18|protein_coding	.	+
chr1	244053787	244057472	ENSG00000179456.12|ENST00000622512.1|ZBTB18|protein_coding	.	+


In [46]:
# introns
intron_gtftk[intron_gtftk['name'].str.contains('ZBTB18')].sort_values(['name', 'start']).reset_index(drop=True)

,chrom,start,end,name,score,strand
0,chr1,244051444,244053787,intron|ENSG00000179456.12|ENST00000358704.4|ZB...,1,+
1,chr1,244048993,244053787,intron|ENSG00000179456.12|ENST00000622512.1|ZB...,1,+


## Check leafcutter produced intron junctions, along with transcript and exon coordiantes

1. exons from leafcutter are 1 based both close format (like VCF or gtf)
2. introns have non-standard coordinates! (**start is 0 based close, but end is non-standard: end appears to be BED-formatted-end + 1**)

In [47]:
# transcripts
# using the same transcript file produced by gtftk
!zcat hg38/use_gtftk/gencode_v43_basic.transcript.bed.gz | awk '$4 ~ /ZBTB18/' | sort -k4 -k2

chr1	244051282	244057476	ENSG00000179456.12|ENST00000358704.4|ZBTB18|protein_coding	.	+
chr1	244048938	244057472	ENSG00000179456.12|ENST00000622512.1|ZBTB18|protein_coding	.	+


In [51]:
# exons
!zcat hg38/use_leafviz/gencode_v43_all_exons.txt.gz |  awk '$5 ~ /ZBTB18/' 

chr1	244048939	244048993	+	ZBTB18
chr1	244053788	244057472	+	ZBTB18
chr1	244051283	244051444	+	ZBTB18
chr1	244053788	244057476	+	ZBTB18


In [53]:
# introns
!zcat hg38/use_leafviz/gencode_v43_all_introns.bed.gz | awk '$4 ~ /ZBTB18/'

chr1	244051444	244053788	ZBTB18	ENSG00000179456.12	+	ENST00000358704.4	1	protein_coding	basic|Ensembl_canonical|MANE_Select|appris_principal_4|CCDS
chr1	244048993	244053788	ZBTB18	ENSG00000179456.12	+	ENST00000622512.1	1	protein_coding	basic|appris_alternative_1


# test

In [1]:
ls ../../hg38/use_gtftk/gencode_v43_basic.intron_by_transcript.bed.gz

../../hg38/use_gtftk/gencode_v43_basic.intron_by_transcript.bed.gz


In [7]:
ls ../../hg38/use_gtftk

gencode_v43_all.exons.bed.gz
gencode_v43_all.genes.bed.gz
gencode_v43_all.intron_by_transcript.bed.gz
gencode_v43_all.transcript.bed.gz
gencode_v43_basic.exons.bed.gz
gencode_v43_basic.intron_by_transcript.bed.gz
gencode_v43_basic.transcript.bed.gz


In [8]:
intron_gtftk = pd.read_csv('../../hg38/use_gtftk/gencode_v43_all.intron_by_transcript.bed.gz', sep='\t', header=None, 
                           names=['chrom', 'start', 'end', 'name', 'score', 'strand'])

In [9]:
intron_gtftk.iloc[1:2]
intron_gtftk.shape

,chrom,start,end,name,score,strand
1,chr1,12721,13220,intron|ENSG00000290825.1|ENST00000456328.2|DDX...,2,+


(1396243, 6)

In [12]:
intron_gtftk[['feature', 'gid', 'tid', 'gene', 'gene_type', 'trpt_type']] = intron_gtftk['name'].str.split('|', expand=True)

In [14]:
intron_gtftk.drop('name', axis=1, inplace=True)

In [15]:
intron_gtftk[1:5]

,chrom,start,end,score,strand,feature,gid,tid,gene,gene_type,trpt_type
1,chr1,12721,13220,2,+,intron,ENSG00000290825.1,ENST00000456328.2,DDX11L2,lncRNA,lncRNA
2,chr1,12057,12178,1,+,intron,ENSG00000223972.6,ENST00000450305.2,DDX11L1,transcribed_unprocessed_pseudogene,transcribed_unprocessed_pseudogene
3,chr1,12227,12612,2,+,intron,ENSG00000223972.6,ENST00000450305.2,DDX11L1,transcribed_unprocessed_pseudogene,transcribed_unprocessed_pseudogene
4,chr1,12697,12974,3,+,intron,ENSG00000223972.6,ENST00000450305.2,DDX11L1,transcribed_unprocessed_pseudogene,transcribed_unprocessed_pseudogene


In [42]:
intron_gtftk.loc[:, 'intron_id'] = intron_gtftk['chrom'].str.cat(intron_gtftk[['start', 'end', 'strand']].apply(lambda x: x.astype(str)), sep=':')

In [43]:
intron_gtftk.head()

,chrom,start,end,score,strand,feature,gid,tid,gene,gene_type,trpt_type,intron_id
0,chr1,12227,12612,1,+,intron,ENSG00000290825.1,ENST00000456328.2,DDX11L2,lncRNA,lncRNA,chr1:12227:12612:+
1,chr1,12721,13220,2,+,intron,ENSG00000290825.1,ENST00000456328.2,DDX11L2,lncRNA,lncRNA,chr1:12721:13220:+
2,chr1,12057,12178,1,+,intron,ENSG00000223972.6,ENST00000450305.2,DDX11L1,transcribed_unprocessed_pseudogene,transcribed_unprocessed_pseudogene,chr1:12057:12178:+
3,chr1,12227,12612,2,+,intron,ENSG00000223972.6,ENST00000450305.2,DDX11L1,transcribed_unprocessed_pseudogene,transcribed_unprocessed_pseudogene,chr1:12227:12612:+
4,chr1,12697,12974,3,+,intron,ENSG00000223972.6,ENST00000450305.2,DDX11L1,transcribed_unprocessed_pseudogene,transcribed_unprocessed_pseudogene,chr1:12697:12974:+


In [54]:
d_test.sort_values('intron_id')

,chrom,start,end,score,strand,feature,gid,tid,gene,gene_type,trpt_type,intron_id
2,chr1,12057,12178,1,+,intron,ENSG00000223972.6,ENST00000450305.2,DDX11L1,transcribed_unprocessed_pseudogene,transcribed_unprocessed_pseudogene,chr1:12057:12178:+
0,chr1,12227,12612,1,+,intron,ENSG00000290825.1,ENST00000456328.2,DDX11L2,lncRNA,lncRNA,chr1:12227:12612:+
3,chr1,12227,12612,2,+,intron,ENSG00000223972.6,ENST00000450305.2,DDX11L1,transcribed_unprocessed_pseudogene,transcribed_unprocessed_pseudogene,chr1:12227:12612:+
4,chr1,12697,12974,3,+,intron,ENSG00000223972.6,ENST00000450305.2,DDX11L1,transcribed_unprocessed_pseudogene,transcribed_unprocessed_pseudogene,chr1:12697:12974:+
1,chr1,12721,13220,2,+,intron,ENSG00000290825.1,ENST00000456328.2,DDX11L2,lncRNA,lncRNA,chr1:12721:13220:+
5,chr1,13052,13220,4,+,intron,ENSG00000223972.6,ENST00000450305.2,DDX11L1,transcribed_unprocessed_pseudogene,transcribed_unprocessed_pseudogene,chr1:13052:13220:+
6,chr1,13374,13452,5,+,intron,ENSG00000223972.6,ENST00000450305.2,DDX11L1,transcribed_unprocessed_pseudogene,transcribed_unprocessed_pseudogene,chr1:13374:13452:+
7,chr1,14501,15004,10,-,intron,ENSG00000227232.5,ENST00000488147.1,WASH7P,unprocessed_pseudogene,unprocessed_pseudogene,chr1:14501:15004:-
8,chr1,15038,15795,9,-,intron,ENSG00000227232.5,ENST00000488147.1,WASH7P,unprocessed_pseudogene,unprocessed_pseudogene,chr1:15038:15795:-
9,chr1,15947,16606,8,-,intron,ENSG00000227232.5,ENST00000488147.1,WASH7P,unprocessed_pseudogene,unprocessed_pseudogene,chr1:15947:16606:-


In [53]:
d_test.groupby('intron_id')[['tid', 'trpt_type']].nunique()

,tid,trpt_type
intron_id,,
chr1:12057:12178:+,1,1
chr1:12227:12612:+,2,2
chr1:12697:12974:+,1,1
chr1:12721:13220:+,1,1
chr1:13052:13220:+,1,1
chr1:13374:13452:+,1,1
chr1:14501:15004:-,1,1
chr1:15038:15795:-,1,1
chr1:15947:16606:-,1,1


In [48]:
help(d_test.groupby('intron_id').nunique)

Help on method nunique in module pandas.core.groupby.generic:

nunique(dropna: 'bool' = True) -> 'DataFrame' method of pandas.core.groupby.generic.DataFrameGroupBy instance
    Return DataFrame with counts of unique elements in each position.
    
    Parameters
    ----------
    dropna : bool, default True
        Don't include NaN in the counts.
    
    Returns
    -------
    nunique: DataFrame
    
    Examples
    --------
    >>> df = pd.DataFrame({'id': ['spam', 'egg', 'egg', 'spam',
    ...                           'ham', 'ham'],
    ...                    'value1': [1, 5, 5, 2, 5, 5],
    ...                    'value2': list('abbaxy')})
    >>> df
         id  value1 value2
    0  spam       1      a
    1   egg       5      b
    2   egg       5      b
    3  spam       2      a
    4   ham       5      x
    5   ham       5      y
    
    >>> df.groupby('id').nunique()
          value1  value2
    id
    egg        1       1
    ham        1       2
    spam       2    